In [1]:
import datetime
import pandas as pd
import time
import requests
import os
import numpy as np

In [2]:
## This class contains the components necessary for running and evaluating a historical simulation.

class SimulationEnv(object):
    
    def __init__(self, wealth, assets, 
                 data_path, start_date, end_date,
                 agent, experts, context_generator):

        self.init_wealth = wealth
        self.wealth = wealth
        self.assets = assets
        self.agent = agent
        self.experts = experts
        self.context_generator = context_generator
        self.data = [pd.read_csv(data_path+s.lower()+".csv", iterator=True, chunksize=1) for s in self.assets]
        
        return
    
    def pre_simulation(self):
        ## Do various stuff here like for select for high-volatility stocks or something
        pass
    
    def run(self):
        while True:
            try:
                self.algorithm.update()
                for expert in self.experts:
                    expert.update()
                self.algorithm.act()
            except StopIteration:
                break
                
                


In [3]:
from abc import ABCMeta, abstractmethod

class Agent(object):
    __metaclass__ =ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    
    @abstractmethod
    def act(self):
        pass
    
class Expert(object):
    __metaclass__=ABCMeta
    
    @abstractmethod
    def __init__(self):
        pass
    
    @abstractmethod
    def update(self):
        pass
    

In [8]:
class Hedge(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        print "hedge update goes here"
        return
    
    def act(self):
        print "hedge action goes here"
        return


class NaiveBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    
class WeightedBuyHold(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    
class ConstantRebalancer(Agent):
    
    def __init__(self):
        return
    
    def update(self):
        return
    
    def act(self):
        return
    
class Dummy(Expert):
    
    def __init__(self, stock):
        self.pick = stock
        return
    
    def update(self):
        pass

In [7]:
h = Hedge()
h.act()

hedge action goes here
